# import

In [28]:
import spikeinterface.full as si
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from probeinterface.plotting import plot_probe
import os
import torch

# set paths and choose probe 

In [29]:

path_ = r"Z:\projects\sequence_squad\revision_data\lars_recordings\ephys\\"

base_recording_paths = []
for q in os.listdir(path_):
    if not 'other_sessions' in q:
        folder = os.path.join(path_,q)
        for q in os.listdir(folder):
            base_recording_paths+=[os.path.join(folder,q)]
            
base_recording_paths

['Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\ephys\\\\ap5\\ap5R_2024-11-20_09-00-27_saline',
 'Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\ephys\\\\ap5\\ap5R_2024-11-21_09-21-32_ap5_ctb647',
 'Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\ephys\\\\ap5\\ap5R_2024-11-25_09-41-07',
 'Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\ephys\\\\ap5\\ap5R_2024-11-26_09-29-29_ap5_488',
 'Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\ephys\\\\dual_strd_hpc\\seq006_2024-11-25_09-37-51',
 'Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\ephys\\\\dual_strd_hpc\\seq006_2024-11-26_09-26-05',
 'Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\ephys\\\\dual_strd_hpc\\seq006_2024-11-27_09-46-17',
 'Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\ephys\\\\dual_strd_hpc\\seq006_2024-11-28_09-48-35',
 'Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\ephys\\\\dual_strd_hpc

# make folder structure

In [30]:
all_recording_paths = []
for i in range(len(base_recording_paths)):

    organised_path = r"Z:\projects\sequence_squad\revision_data\organised_data\animals\\"

    mouse_id = base_recording_paths[i].split('\\')[-1].split('_')[0]
    print(mouse_id)
    date_ = base_recording_paths[i].split('\\')[-1].split('_')[1]
    print(date_)
    #reverse the date
    date_ = '-'.join(date_.split('-')[::-1])

    if not os.path.isdir(organised_path+mouse_id+'_implant1'):
        print('adding mouse file')
        os.makedirs(organised_path+mouse_id+'_implant1')
    else:
        print('mouse file already exists')
        
    #work out what rnum will be
    make_folder = False
    if len(os.listdir(organised_path+mouse_id+'_implant1\\')) == 0:
        r_num = '1'
        make_folder = True
    else:
        r_num = str(int(os.listdir(organised_path+mouse_id+'_implant1\\')[-1].split('_')[0][-1]) + 1)
        for item in os.listdir(organised_path+mouse_id+'_implant1\\'):
            if not date_ in item:
                make_folder = True
            else:
                make_folder = False
                r_num = item.split('_')[0].split('ing')[-1]
                break
        
    # make recording dir
    if make_folder == True:
        print('**making new recording folder')
        recording_path = organised_path+mouse_id+'_implant1' + '\\recording' + r_num + '_' + date_ + '\\'
        os.makedirs(recording_path)
        # make folder structure 
        if not os.path.isdir(recording_path + 'ephys'):
            os.makedirs(recording_path + 'ephys')
        if not os.path.isdir(recording_path + 'video/tracking/'):
            os.makedirs(recording_path + 'video/tracking/')
        if not os.path.isdir(recording_path + 'behav_sync/'):
            os.makedirs(recording_path + 'behav_sync/') 
            

    else:
        print('recording folder already exists')
        recording_path = organised_path+mouse_id+'_implant1' + '\\recording' + r_num + '_' + date_ + '\\'
    all_recording_paths+=[recording_path + 'ephys']




  


ap5R
2024-11-20
mouse file already exists
recording folder already exists
ap5R
2024-11-21
mouse file already exists
recording folder already exists
ap5R
2024-11-25
mouse file already exists
recording folder already exists
ap5R
2024-11-26
mouse file already exists
recording folder already exists
seq006
2024-11-25
mouse file already exists
recording folder already exists
seq006
2024-11-26
mouse file already exists
recording folder already exists
seq006
2024-11-27
mouse file already exists
recording folder already exists
seq006
2024-11-28
mouse file already exists
recording folder already exists
seq007
2024-11-29
mouse file already exists
recording folder already exists
seq008
2024-11-11
mouse file already exists
recording folder already exists
seq008
2024-11-12
mouse file already exists
recording folder already exists
seq008
2024-11-13
mouse file already exists
recording folder already exists
seq008
2024-11-15
mouse file already exists
recording folder already exists
ap5lr
2024-11-15
mou

# chose which data to process - ie. data that hasnt yet been processed - and create save paths

In [31]:

process_base_paths  = []
out_paths = []
missing_probe = []
for index,path in enumerate(all_recording_paths):
    if not os.path.isfile(path+ r'\probeA\unit_info.txt'):
        # if no kilosort folder then add base path to process list
        process_base_paths += [base_recording_paths[index]]
        out_paths += [path]
        missing_probe += ['A']

    if not 'ap5' in path:
        if not os.path.isfile(path+ r'\probeB\unit_info.txt'):
            # if no kilosort folder then add base path to process list
            process_base_paths += [base_recording_paths[index]]
            out_paths += [path]
            missing_probe += ['B']

In [44]:
process_base_paths

['Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\ephys\\\\dual_strd_hpc\\seq007_2024-11-29_10-29-42',
 'Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\ephys\\\\learning\\ap5L_2024-11-18_13-59-54',
 'Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\ephys\\\\learning\\seq007_2024-11-18_08-51-40',
 'Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\ephys\\\\learning\\seq007_2024-11-19_14-14-27',
 'Z:\\projects\\sequence_squad\\revision_data\\lars_recordings\\ephys\\\\learning\\seq007_2024-11-20_13-27-15']

# kilosort loop

In [45]:

for ind,base_folder in enumerate(process_base_paths[2::]):
    ind = ind + 2
    
    # clear gpu memory cache each loop to avoid memory issues
    torch.cuda.empty_cache()
    print(str(ind) + 'out of' + str(len(process_base_paths)))
    print('*********** PROCESSING: ***********')
    print(base_folder)
    out_path = out_paths[ind]
    
    print(missing_probe[ind])
    print(out_path)


    # extract the stream names (each np processor)
    stream_names, stream_ids = si.get_neo_streams('openephysbinary', base_folder)
    print(stream_names)

    # chose probe id, DEAL WITH PROBE A/B stuff
    ksort = False
    for stream_i, stream in enumerate(stream_names):
        if 'Probe' + missing_probe[ind] in stream:
            if not 'LFP' in stream:
                Probe_id = stream_names[stream_i]
                out_path = out_path + '\\probe' + missing_probe[ind] + '\\'
                out_path_object = Path(out_path)
                ksort = True
                if not os.path.isdir(out_path):
                    os.makedirs(out_path)
                break

    if ksort == True:
        
        # load in data
        raw_rec = si.read_openephys(base_folder,stream_name = Probe_id,load_sync_channel=False)
        raw_rec.get_probe().to_dataframe()
    
        # plot probe
        fig, axs = plt.subplots(figsize=(1, 100))
        probe = raw_rec.get_probe()
        plot_probe(probe, ax = axs)
        plt.savefig(out_path + 'probe_map.png')
        plt.close()
            
        
        # Preprocess the recording¶
        # Let’s do something similar to the IBL destriping chain (See :ref:ibl_destripe) to preprocess the data but:
        # instead of interpolating bad channels, we remove then.
        # instead of highpass_spatial_filter() we use common_reference()
        
        rec1 = si.highpass_filter(raw_rec, freq_min=400.)
        bad_channel_ids, channel_labels = si.detect_bad_channels(rec1)
        rec2 = rec1.remove_channels(bad_channel_ids)
        print('bad_channel_ids', bad_channel_ids)
    
        rec3 = si.phase_shift(rec2)
        rec4 = si.common_reference(rec3, operator="median", reference="global")
        rec = rec4
        
        ## save out the preprocessed binary
        job_kwargs = dict(n_jobs=40, chunk_duration='1s', progress_bar=True)
        rec = rec.save(folder=out_path_object / 'preprocess', format='binary', **job_kwargs, overwrite = True)
        
        # here we use static plot using matplotlib backend
        fig, axs = plt.subplots(ncols=3, figsize=(20, 10))
    
        si.plot_traces(rec1, backend='matplotlib',  clim=(-50, 50), ax=axs[0])
        si.plot_traces(rec4, backend='matplotlib',  clim=(-50, 50), ax=axs[1])
        si.plot_traces(rec, backend='matplotlib',  clim=(-50, 50), ax=axs[2])
        for i, label in enumerate(('filter', 'cmr', 'final')):
            axs[i].set_title(label)
        plt.savefig(out_path + 'preprocessing_destriping_common_ref.png')
        plt.close()
        
        
        # plot some channels
        fig, ax = plt.subplots(figsize=(20, 10))
        some_chans = rec.channel_ids[[100, 150, 200, ]]
        si.plot_traces({'filter':rec1, 'cmr': rec4}, backend='matplotlib', mode='line', ax=ax, channel_ids=some_chans)
        plt.savefig(out_path + 'example_chans.png')
        plt.close()
        
        # check noise 
        # we can estimate the noise on the scaled traces (microV) or on the raw one (which is in our case int16).
        noise_levels_microV = si.get_noise_levels(rec, return_scaled=True)
        noise_levels_int16 = si.get_noise_levels(rec, return_scaled=False)
        
        fig, ax = plt.subplots()
        _ = ax.hist(noise_levels_microV, bins=np.arange(5, 30, 2.5))
        ax.set_xlabel('noise  [microV]')
        plt.savefig(out_path + 'noise_level.png')
        plt.close()
        
        
        # check default params for kilosort4
        params_kilosort4 = si.get_default_sorter_params('kilosort4')
        params_kilosort4['delete_recording_dat'] = False
    
        # # run kilosort4 with drift correction (set as True in the params)
        sorting = si.run_sorter('kilosort4', rec, output_folder=out_path_object / 'kilosort4_output',
                                docker_image=False, verbose=True, **params_kilosort4, remove_existing_folder = True)
        
        
        ######################################################################################
        # load back in to check quality
        sorting = si.read_sorter_folder(out_path_object / 'kilosort4_output')
        
        analyzer = si.create_sorting_analyzer(sorting, rec, sparse=True, format="memory")
        
        analyzer.compute("random_spikes", method="uniform", max_spikes_per_unit=500)
        analyzer.compute("waveforms",  ms_before=1.5,ms_after=2., **job_kwargs)
        analyzer.compute("templates", operators=["average", "median", "std"])
        analyzer.compute("noise_levels")
    
        analyzer_saved = analyzer.save_as(folder=out_path_object / "analyzer", format="binary_folder")
    
        metric_names=['firing_rate', 'presence_ratio', 'snr', 'isi_violation', 'amplitude_cutoff']
    
        metrics = si.compute_quality_metrics(analyzer, metric_names=metric_names)
    
        amplitude_cutoff_thresh = 0.1
        isi_violations_ratio_thresh = 1
        presence_ratio_thresh = 0.9
    
        our_query = f"(amplitude_cutoff < {amplitude_cutoff_thresh}) & (isi_violations_ratio < {isi_violations_ratio_thresh}) & (presence_ratio > {presence_ratio_thresh})"
    
        keep_units = metrics.query(our_query)
        keep_unit_ids = keep_units.index.values
    
        analyzer_clean = analyzer.select_units(keep_unit_ids, folder=out_path_object / 'analyzer_clean', format='binary_folder')
    
        # export spike sorting report to a folder
        si.export_report(analyzer_clean, out_path_object / 'report', format='png')

        ### SAVE OUT A TXT FILE WITH NUMBER OF UNITS DATA ON IT
        # Open a file in write mode
        file_path = out_path + 'unit_info.txt'
        with open(file_path, "w") as file:
            # Use the file argument to save print output to the file
            print('Kilosort output:',file = file)
            print(sorting, file=file)
            file.write("\n")
            print('"good" units:',file = file)
            print(analyzer_clean, file=file)

        print('DONE!')
    else:
        print('Already processed or no probe B!')


    
                
    

2out of5
*********** PROCESSING: ***********
Z:\projects\sequence_squad\revision_data\lars_recordings\ephys\\learning\seq007_2024-11-18_08-51-40
A
Z:\projects\sequence_squad\revision_data\organised_data\animals\\seq007_implant1\recording1_18-11-2024\ephys
['Record Node 103#NI-DAQmx-102.PXIe-6341', 'Record Node 103#Neuropix-PXI-100.ProbeA', 'Record Node 103#Neuropix-PXI-100.ProbeB']


KeyboardInterrupt: 

In [12]:
# # find peaks

# from spikeinterface.sortingcomponents.peak_detection import detect_peaks
# from spikeinterface.sortingcomponents.peak_localization import localize_peaks
# break
# job_kwargs = dict(n_jobs=40, chunk_duration='1s', progress_bar=True)
# peaks = detect_peaks(rec,  method='locally_exclusive', noise_levels=noise_levels_int16,
#                      detect_threshold=5, radius_um=50., **job_kwargs)

# peak_locations = localize_peaks(rec, peaks, method='center_of_mass', radius_um=50., **job_kwargs)



SyntaxError: 'break' outside loop (3566307101.py, line 5)

In [ ]:
# # check for drifts
# fs = rec.sampling_frequency
# fig, ax = plt.subplots(figsize=(10, 8))
# ax.scatter(peaks['sample_ind'] / fs, peak_locations['y'], color='k', marker='.',  alpha=0.002)

# Load back in to check quality 

In [ ]:

analyzer

C:\Users\Emmett\anaconda3\envs\si_env\Lib\site-packages\spikeinterface\core\job_tools.py:103: UserWarning: `n_jobs` is not set so parallel processing is disabled! To speed up computations, it is recommended to set n_jobs either globally (with the `spikeinterface.set_global_job_kwargs()` function) or locally (with the `n_jobs` argument). Use `spikeinterface.set_global_job_kwargs?` for more information about job_kwargs.
  warnings.warn(


estimate_sparsity:   0%|          | 0/14682 [00:00<?, ?it/s]

SortingAnalyzer: 384 channels - 373 units - 1 segments - memory - sparse - has recording
Loaded 0 extensions

compute_waveforms:   0%|          | 0/14682 [00:00<?, ?it/s]

SortingAnalyzer: 384 channels - 356 units - 1 segments - memory - sparse - has recording
Loaded 4 extensions: random_spikes, waveforms, templates, noise_levels

In [43]:
# quality metrics 

C:\Users\Emmett\anaconda3\envs\si_env\Lib\site-packages\spikeinterface\qualitymetrics\misc_metrics.py:908: UserWarning: Some units have too few spikes : amplitude_cutoff is set to NaN
  warnings.warn(f"Some units have too few spikes : amplitude_cutoff is set to NaN")


,firing_rate,presence_ratio,snr,isi_violations_ratio,isi_violations_count,amplitude_cutoff
0,2.06766,0.983607,2.156246,5.724791,1078,0.00309
1,1.276477,0.897541,1.117945,10.729112,770,0.0067
2,0.781851,1.0,4.024969,0.408548,11,0.005223
3,1.764291,1.0,1.848011,3.078016,422,0.001179
4,2.985189,1.0,3.531435,0.030573,12,0.000694
...,...,...,...,...,...,...
351,1.075003,1.0,4.915325,0.176816,9,0.001501
352,1.342204,1.0,2.708582,0.919993,73,0.000756
353,2.283096,1.0,2.799555,0.688189,158,0.000905
354,7.896229,1.0,4.028388,0.102685,282,0.083223


(amplitude_cutoff < 0.1) & (isi_violations_ratio < 1) & (presence_ratio > 0.9)


123

C:\Users\Emmett\anaconda3\envs\si_env\Lib\site-packages\spikeinterface\core\job_tools.py:103: UserWarning: `n_jobs` is not set so parallel processing is disabled! To speed up computations, it is recommended to set n_jobs either globally (with the `spikeinterface.set_global_job_kwargs()` function) or locally (with the `n_jobs` argument). Use `spikeinterface.set_global_job_kwargs?` for more information about job_kwargs.
  warnings.warn(


export_report(): spike_amplitudes will not be exported. Use sorting_analyzer.compute('spike_amplitudes') if you want to include them.
export_report(): correlograms will not be exported. Use sorting_anlyzer.compute('correlograms') if you want to include them.
